## Computing lensing C_ell(ell) for reasonable redshift distribution of sources and parameter values


To get started with computing and also understanding the C_ells: once you have CCL installed, try to use it to compute and plot lensing C_ell as a function of ell for any reasonable redshift distribution of sources and any reasonable parameter values (combining equation 16 and equation 5 in the first paper above by Krause et al.). Use CCL to get the basic building blocks of this like the matter power spectrum and comoving distances, but don't just use the functionality to directly get the C_ells, do the integration numerically yourself using numpy / scipy. You should however use the built in CCL functionality to check you're getting it right and debug. You can find examples for using CCL that will help here: https://github.com/LSSTDESC/CCLX (you can also steal a redshift distribution to use from where they construct one in those examples notebooks).

Eq 1: $C^{ij}_{\mathcal{A}\mathcal{B}}(\ell) = \int d\chi \frac{W^{i}_{\mathcal{A}}(\chi)W^{j}_{\mathcal{B}}(\chi) }{\chi^2}P_{\mathcal{A}\mathcal{B}} ( k = \frac{\ell + 0.5}{\chi}, z) $

Where:
- i,j denote redshift bins
- Subscripts $\mathcal{A}\mathcal{B} \in \{\delta_g, \kappa\}$ stand for specific E- and B- modes (kappa) or for specific observed galaxy number densities (delta)
- C is the angular power spectrum
- $P_{\delta}$ is three-dimensional matter power spectrum
- $\chi$ is comoving distance

Eq 4: $W^{i}_{\kappa,g}(\chi) = \frac{3\Omega_{m}H_{0}^{2}}{2c^2} \int_{\chi}^{\infty}d\chi'n^{i}_{g}(\chi')\frac{\chi}{a(\chi)}\frac{\chi' - \chi}{\chi'}$

Where W is the tomographic lens efficiency

We are looking for shear power spectra, so:

$C^{ij}_{EE}(\ell) = C^{ij}_{\kappa_s\kappa_s}(\ell) + C^{ij}_{\kappa_s I_E}(\ell) + C^{ji}_{\kappa_s I_E}(\ell) + C^{ij}_{I_E I_E}(\ell)$

or 

$C^{ij}_{BB}(\ell) = C^{ij}_{I_B I_B}(\ell)$

$I_{E/B}$ stand for the E/B mode decomposition of IA, and we will ignore these components for our analysis.
$\kappa_s$ stands for the tomographic convergence field of the source galaxy (?)

In [ ]:
# defining all functions to find C_ell

# define the reasonable redshift distribution from github
def redshiftdist(z):                                                       
    i_lim = 26.                                                            # Limiting i-band magnitude
    z0 = 0.0417 * i_lim - 0.744
    pz = 1. / (2. * z0) * (z / z0)**2. * np.exp(-z / z0)                   # Redshift distribution, p(z)
    return pz                                                              # to check for normalized pz print(scipy.integrate.simps(pz, z))

# solves equation 4
# note: z here is an array not a single value. returns array

def integrandWkg(x,cosmo,chiz,a):
    ax = 1/(1 + x)
    chix = ccl.comoving_radial_distance(cosmo,ax)
    dchidx = c/(H0*ccl.h_over_h0(cosmo,ax))
    px = redshiftdist(x)
    integrand = dchidx*px*chiz*(chix - chiz)/(a*chix)
    return integrand

def Wkappag(z, Omega_m, H0, c, cosmo):
    zmax = 1090.3                                                          # z at horizon from Planck 2018
    a = 1/(1 + z)
    chiz = ccl.comoving_radial_distance(cosmo,a)                           # units of Mpc
    y,err = scipy.integrate.quad(integrandWkg, z, zmax, args=(cosmo,chiz, a))
    Wkg = y*3*Omega_m*(H0**2)/(2*c**2)
    return Wkg

# function that finds C for a given  ell, z. A, B, i, j can be changed by manually changing quantities in the function
# Note: if we assume \LambdaCDM cosmology (or some cosmology) don't need function h_over_h0. 
# Can find chi(z) and z(chi) manually

def integrandC_ell(z,cosmo,H0,c,Omega_m,ell):
    a = 1/(1 + z)
    chiz = ccl.comoving_radial_distance(cosmo,a)
    dchidz = c/(H0*ccl.h_over_h0(cosmo,a))
    Wkg = Wkappag(z, Omega_m, H0, c, cosmo)                               # find tomographic lens efficiency
    
    MatterPowerSpec_lz = ccl.power.nonlin_matter_power(cosmo, (ell + 0.5)/chiz , z)
    MatterPowerSpec_lz = np.transpose(MatterPowerSpec_lz)
    
    integrand = dchidz*Wkg*Wkg*MatterPowerSpec_lz/(chiz**2)
    return integrand

def findC_ell(cosmo,H0,c,Omega_m,ell,z,bz):
                                                                          # find C_ell
    C_ell = np.zeros(len(ell))
    for i in range(len(ell)):
        C_ell[i] = scipy.integrate.quad(integrandC_ell, 0.001, 5, args= (cosmo,H0,c,Omega_m,ell[i]))
    return C_ell

In [ ]:
#bits of the code come from CCLX/Example MCMC analysis on GitHub

import emcee
import matplotlib.pyplot as plt 
import pyccl as ccl
import numpy as np
import scipy
%matplotlib inline

# for nicer plots http://seaborn.pydata.org -- optional
# if you don't want to use it, you can comment this and the next line

import seaborn as sns; sns.set_style('darkgrid')
sns.set_context("talk")

# Program starts here

In [ ]:
# define cosmological parameters
cosmo = ccl.Cosmology(Omega_c=0.27, 
                      Omega_b=0.045, 
                      h=0.67, 
                      n_s=0.96,  
                      sigma8=0.83, 
                      transfer_function='bbks')

# Note: find way of extracting these params directly from cosmo
c = 3e5                                                                  # km/s
H0 = 0.67e2                                                              # km/s/Mpcs
Omega_m = 0.27 + 0.045 

In [ ]:
# create an angular shear C_ell manually with our functions above
# Note: define ell from github 
# np.seterr(divide='ignore', invalid='ignore') -- in integrals ignores bits when/0

ell = np.arange(2, 2000)
delta_ell = ell[1] - ell[0]
z = np.linspace(0.001, 3., int(1e4))                                     # z =/= 0 otherwise get inf when calculating chi

# define bias distribution - assume no galaxy bias (b^i = 1)
bz = np.ones_like(z)

# calculate C_ell
manualdata = findC_ell(cosmo,H0,c,Omega_m,ell,z,bz)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
#create an angular shear C_ell with pyccl from github

i_lim = 26.                                                              # Limiting i-band magnitude
z0 = 0.0417 * i_lim - 0.744
Ngal = 46. * 100.31 * (i_lim - 25.)                                      # Normalisation, galaxies/arcmin^2
pz = 1. / (2. * z0) * (z / z0)**2. * np.exp(-z / z0)                     # Redshift distribution, p(z)
dNdz = Ngal * pz                                                         # Number density distribution

tracer1 = ccl.WeakLensingTracer(cosmo, dndz=(z, dNdz))
tracer2 = ccl.WeakLensingTracer(cosmo, dndz=(z, dNdz))
data    = ccl.angular_cl(cosmo, tracer1, tracer2, ell)                   # C_ell - uses non-linear matter power spectrum stored in `cosmo`

In [ ]:
# Plots of C_ell

plt.plot(ell, data, "b")

plt.plot(ell,manualdata, "r")
plt.xscale('log')
plt.yscale('log')
plt.xlabel(r'$\ell$')
plt.ylabel(r'$C_\ell$')
plt.show()

In [ ]:
print("chi_h = ", c/H0)
print("chi_max = ", ccl.comoving_radial_distance(cosmo,1/(1+3)))
print("a_h = ", ccl.scale_factor_of_chi(cosmo, c/H0))
print("z_h = ", 1/ccl.scale_factor_of_chi(cosmo, c/H0) -1)

chi_h =  4477.611940298508
chi_max =  6543.4031147575815
a_h =  0.4025993859593187
z_h =  1.4838587312228206


- (Not useful now) q are the radial weight functions:

Eq 2: $q^i_\kappa (\chi) = \frac{3H_0^2 \Omega_m \chi}{2a(\chi)} \int^{\chi_h}_\chi d\chi' \frac{\chi' - \chi}{\chi} n^i_\kappa (z(\chi')) \frac{dz}{d\chi'}$

Eq 3: $q^i_{\delta_g} (\chi) = b^i n^i_\delta (z(\chi)) \frac{dz}{d\chi}$



In [ ]:
# bits of code I don't use now

# solves equation 2
# z is a single value not an array
def qkappaz(z, cosmo, c, H0, Omega_m):
    #define useful quantities for integral
    a = 1/(1 + z)
    chiz = ccl.comoving_radial_distance(cosmo,a)
    zh = 1/ccl.scale_factor_of_chi(cosmo, c/H0) - 1
    pz =  redshiftdist(z)
    
    #find integral
    x = np.linspace(0.01,zh,int(1e4)) #variable we integrate over
    chix = ccl.comoving_radial_distance(cosmo,1/(1 + x))
    integrand = pz*(chix - chiz)/chiz
    I = scipy.integrate.simps(integrand, x)
    qk = I*3*(H0**2)*Omega_m*chiz/(2*a*chiz)
    return qk

#solves equation 3
#z is a single value not an array
def qdelgz(z, cosmo, c, H0, Omega_m, bz):
    pz = redshiftdist(z)
    a = 1/(1 + z)
    qd = ccl.h_over_h0(cosmo,a)*pz*bz*H0/c
    return qd